In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro, wilcoxon
import numpy as np

# Load and clean the data
file_path = '無題のフォーム（回答） - シート1 (1).csv'
data = pd.read_csv(file_path)
data.columns = data.columns.str.strip().str.replace('\n', '').str.replace(' ', '')

# Extract relevant columns
data_cleaned = data[['実験手順', '実験番号', 'どれくらい酔ったか？', 'どれくらいターゲットを取得するのが大変だったか？',
                     'どれくらい視野外物体の区別をするのが難しかったか？', '精神的にしんどかったですか？', 'どれくらい体験として楽しかったか？']]
# print(data_cleaned)
data_cleaned.columns = ['Condition', 'Experiment', 'Dizziness', 'Target Difficulty', 'Object Distinction Difficulty', 
                        'Mental Strain', 'Fun Experience']
output_file = 'data-cleaned-all.csv'
data_cleaned.to_csv(output_file, index=False)
# Get unique experiment numbers
experiment_numbers = data_cleaned['Experiment'].unique()
# print(experiment_numbers)
print(data_cleaned.isnull().sum())  # 各列の欠損値の数を確認
print(data_cleaned[data_cleaned.isnull().any(axis=1)])  # 欠損値を含む行を表示

# Metrics for evaluation
metrics = ['Dizziness', 'Target Difficulty', 'Object Distinction Difficulty', 'Mental Strain', 'Fun Experience']

# Wilcoxon signed-rank test for non-parametric comparison
wilcoxon_results = []
for experiment in experiment_numbers:
    experiment_data = data_cleaned[data_cleaned['Experiment'] == experiment]
    for metric in metrics:
        sound_on = experiment_data[experiment_data['Condition'] == '音あり'][metric].dropna()
        sound_off = experiment_data[experiment_data['Condition'] == '音無し'][metric].dropna()
        # print(sound_on)
        # print(sound_off)

            # Perform Wilcoxon signed-rank test if data sizes match
        if len(sound_on) == len(sound_off):
            diff = sound_on - sound_off
            # print(diff)
            if diff.abs().sum() == 0:  # 全ての差がゼロの場合を検出
                wilcoxon_results.append({
                    'Experiment': experiment,
                    'Metric': metric,
                    'W-Statistic': None,
                    'p-Value': None,
                    'Effect Size (r)': None,
                    'Note': 'All differences are zero'
                })
            else:
                stat, p = wilcoxon(sound_on, sound_off)
                # Calculate effect size (r = Z / sqrt(N))
                N = len(sound_on)
                effect_size = stat / np.sqrt(N) if N > 0 else None
                wilcoxon_results.append({
                    'Experiment': experiment,
                    'Metric': metric,
                    'W-Statistic': stat,
                    'p-Value': p,
                    'Effect Size (r)': effect_size
                })
        else:
            wilcoxon_results.append({
                'Experiment': experiment,
                'Metric': metric,
                'W-Statistic': None,
                'p-Value': None,
                'Effect Size (r)': None,
                'Note': 'Unequal sample sizes'
            })


# Convert results to a DataFrame
wilcoxon_results_df = pd.DataFrame(wilcoxon_results)
# 正規性の検定結果をCSVファイルとして保存
output_file = 'wilcoxon_results-all.csv'
wilcoxon_results_df.to_csv(output_file, index=False)

print(f"Wilcoxon検定結果が {output_file} に保存されました。")

# Output the results for inspection
# import ace_tools as tools; tools.display_dataframe_to_user(name="Wilcoxon Signed-Rank Test Results with Effect Sizes", dataframe=wilcoxon_results_df)


Condition                        0
Experiment                       0
Dizziness                        0
Target Difficulty                0
Object Distinction Difficulty    0
Mental Strain                    0
Fun Experience                   0
dtype: int64
Empty DataFrame
Columns: [Condition, Experiment, Dizziness, Target Difficulty, Object Distinction Difficulty, Mental Strain, Fun Experience]
Index: []
Wilcoxon検定結果が wilcoxon_results-all.csv に保存されました。


In [11]:
import pandas as pd
from scipy.stats import wilcoxon
import numpy as np

# Load and clean the data
file_path_new = '無題のフォーム（回答） - シート1 (1).csv'
data_new = pd.read_csv(file_path_new)
data_new.columns = data_new.columns.str.strip().str.replace('\n', '')
data_cleaned = data_new[['実験番号', '実験手順', 'どれくらい酔ったか？', 
                         'どれくらいターゲットを取得するのが大変だったか？', 
                         'どれくらい視野外物体の区別をするのが難しかったか？', 
                         '精神的にしんどかったですか？', 
                         'どれくらい体験として楽しかったか？']]

data_cleaned.columns = ['Experiment', 'Condition', 'Dizziness', 'Target Difficulty', 
                        'Object Distinction Difficulty', 'Mental Strain', 'Fun Experience']

# Unique experiments and metrics for evaluation
experiments = data_cleaned['Experiment'].unique()
metrics = ['Dizziness', 'Target Difficulty', 'Object Distinction Difficulty', 'Mental Strain', 'Fun Experience']

# Perform Wilcoxon signed-rank test for each experiment and metric
results = []

for experiment in experiments:
    experiment_data = data_cleaned[data_cleaned['Experiment'] == experiment]
    for metric in metrics:
        sound_on = experiment_data[experiment_data['Condition'] == '音あり'][metric].dropna()
        sound_off = experiment_data[experiment_data['Condition'] == '音無し'][metric].dropna()

        # Perform Wilcoxon signed-rank test if sample sizes are equal
        if len(sound_on) == len(sound_off):
            # Calculate differences and check if all differences are zero
            differences = sound_on.values - sound_off.values
            if not all(differences == 0):
                stat, p = wilcoxon(sound_on, sound_off)
                # Calculate effect size (r = Z / sqrt(N))
                N = len(sound_on)
                effect_size = stat / np.sqrt(N) if N > 0 else None
                results.append({'Experiment': experiment, 'Metric': metric, 'W-Statistic': stat, 
                                'p-Value': p, 'Effect Size (r)': effect_size})
            else:
                results.append({'Experiment': experiment, 'Metric': metric, 'W-Statistic': None, 
                                'p-Value': None, 'Effect Size (r)': None, 'Note': 'All differences are zero'})
        else:
            results.append({'Experiment': experiment, 'Metric': metric, 'W-Statistic': None, 
                            'p-Value': None, 'Effect Size (r)': None, 'Note': 'Unequal sample sizes'})

# Convert results to a DataFrame and display
results_df = pd.DataFrame(results)
# Convert results to a DataFrame
# results_df = pd.DataFrame(wilcoxon_results)
# 正規性の検定結果をCSVファイルとして保存
output_file = 'wilcoxon_results-all.csv'
results_df.to_csv(output_file, index=False)

print(f"Wilcoxon検定結果が {output_file} に保存されました。")


Wilcoxon検定結果が wilcoxon_results-all.csv に保存されました。


c:\Users\takaharayota\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)
